In [ ]:
#Канонический фаторный анализ методом максимального правдоподобия
import pandas as pd
import numpy as np
from itertools import pairwise
data = pd.read_excel('LAB_DATA_MSM.xlsx', sheet_name = 'Котировки')
columns = [3, 4, 6, 13, 14, 16, 17, 20]
df_prices = np.array(data[columns])[1:,:].T

def rel_increment(x):#относительное приращение
  l = pairwise(x)
  return [(i[1]-i[0])/i[0] for i in l]
data_increment =np.array( [rel_increment(i) for i in df_prices])

data_increment = np.array([(i-i.mean())/np.std(i) for i in data_increment])
n = data_increment.shape[1]


In [ ]:
#Оценим максимально возможное количество обощенных факторов
k = len(data_increment)#8
m_max = 4#(k-m)^2>(k+m)
#Примем число обощенных факторов
m = 1

In [ ]:
#Найдем оценки факторных нагрузок и дисперсий характерных факторов для факторной модели
A = np.cov(data_increment,bias=True)
def max_cor(num1, A_matrix):
    """Максимальная корреляция переменной num1 с другими"""
    row = A_matrix[num1, :].copy()  # Копируем строку
    row[num1] = 0  # Обнуляем диагональ
    return np.max(np.abs(row))  # Берем абсолютное значение

# Вычисляем максимальные корреляции
r_i_mat = []
for i in range(k):
    r_i_mat.append(max_cor(i, A))
r_i_mat = np.array(r_i_mat)
pd.DataFrame(A)

,0,1,2,3,4,5,6,7
0,1.000000,0.275313,0.190738,0.052724,0.087929,0.349217,0.106634,0.130206
1,0.275313,1.000000,0.163025,0.151212,0.218834,0.374313,0.109374,0.172881
2,0.190738,0.163025,1.000000,0.223167,0.359416,0.317377,0.393247,0.128869
3,0.052724,0.151212,0.223167,1.000000,0.129302,0.168850,0.210165,0.685968
4,0.087929,0.218834,0.359416,0.129302,1.000000,0.200681,0.234732,0.116608
5,0.349217,0.374313,0.317377,0.168850,0.200681,1.000000,0.403456,0.173488
6,0.106634,0.109374,0.393247,0.210165,0.234732,0.403456,1.000000,0.211035
7,0.130206,0.172881,0.128869,0.685968,0.116608,0.173488,0.211035,1.000000


In [ ]:
#Задаем начальное приближение для диагональной матрицы хаарктерных факторов
D_i = np.var(data_increment, axis=1, ddof=1)
mat = (np.ones(k)-np.abs(r_i_mat))*D_i
E_mat0 = np.zeros((k, k))
for i in range(k):
  E_mat0[i, i]=mat[i]
pd.DataFrame(E_mat0)

,0,1,2,3,4,5,6,7
0,0.654751,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,0.000000,0.629502,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.610453,0.000000,0.00000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.315947,0.00000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.64449,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.00000,0.600181,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.600181,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.315947


In [ ]:
#Находим собственные векторы следующей матрицы
E = (A - E_mat0)@np.linalg.inv(E_mat0)
eig_val, eig_vec = np.linalg.eig(E)
eig_vec = eig_vec.T
print(eig_val)
pd.DataFrame(eig_vec)

[ 5.0157976   2.04291893  0.77149841  0.37170702 -0.25296299 -0.17108622
  0.13629537  0.05392906]


,0,1,2,3,4,5,6,7
0,-0.168793,-0.230927,-0.272464,-0.565921,-0.204147,-0.284850,-0.289989,-0.562822
1,-0.340784,-0.327615,-0.414926,0.249056,-0.340916,-0.482680,-0.356960,0.256356
2,-0.531696,-0.489895,0.400667,0.080739,0.317396,-0.208819,0.391938,-0.110851
3,-0.152457,0.410064,0.070514,0.047353,0.684579,-0.305053,-0.488274,-0.035682
4,0.365016,0.240505,-0.327959,0.185689,0.055072,-0.576388,0.536724,-0.208697
5,0.142392,-0.431165,-0.528174,0.253248,0.429169,0.453578,-0.106487,-0.222399
6,-0.685000,0.464069,-0.401387,-0.018873,-0.046026,0.269212,0.281203,-0.016700
7,0.125128,-0.233813,-0.389529,-0.474293,0.488406,-0.115332,0.278377,0.472919


In [ ]:
#Нормировка соственных векторов
c = (eig_vec[0].T@np.linalg.inv(E_mat0)@eig_vec[0])
eig_vec[0]= eig_vec[0]*np.sqrt(np.abs(eig_val[0]/c))



In [ ]:
E_mat  = np.zeros((k,k))
matrix = eig_vec[0]**2
diag_A = np.diag(A)
_mat = diag_A - matrix
for i in range(k):
  E_mat[i,i] = _mat[i]
pd.DataFrame(E_mat)

,0,1,2,3,4,5,6,7
0,0.945164,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.897364,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.000000,0.000000,0.85712,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.00000,0.383597,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.00000,0.000000,0.919788,0.000000,0.000000,0.00000
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.843835,0.000000,0.00000
6,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.838149,0.00000
7,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.39033


In [ ]:
np.abs(np.trace(E_mat)-np.trace(E_mat0))

np.float64(1.7038957420402179)

In [ ]:
def func(E_mat0, A ):
  eig_val, eig_vec = np.linalg.eig((A - E_mat0)@np.linalg.inv(E_mat0))
  eig_vec = eig_vec.T

  #Нормировка соственных векторов
  c = (eig_vec[0].T@np.linalg.inv(E_mat0)@eig_vec[0])
  eig_vec[0]= eig_vec[0]*np.sqrt(np.abs(eig_val[0]/c))
  E_mat  = np.zeros((k,k))
  matrix = eig_vec[0]**2
  diag_A = np.diag(A)
  _mat = diag_A - matrix
  for i in range(k):
    E_mat[i,i] = _mat[i]
  return E_mat0, E_mat

D_i = np.var(data_increment, axis=1, ddof=1)
mat = (np.ones(k)-np.abs(r_i_mat))*D_i
E_mat0 = np.zeros((k, k))
for i in range(k):
  E_mat0[i, i]=mat[i]
E_mat0, E_mat = func(E_mat0, A )

while np.abs(np.trace(E_mat)-np.trace(E_mat0))>0.1:
  E_mat0 = E_mat
  E_mat0, E_mat = func(E_mat0, A )
  print(np.abs(np.trace(E_mat)-np.trace(E_mat0)))

pd.DataFrame(E_mat)

0.16558173553720668
0.016618136519189086


,0,1,2,3,4,5,6,7
0,0.96253,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.00000,0.923589,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
2,0.00000,0.000000,0.895011,0.00000,0.000000,0.000000,0.000000,0.000000
3,0.00000,0.000000,0.000000,0.37822,0.000000,0.000000,0.000000,0.000000
4,0.00000,0.000000,0.000000,0.00000,0.942472,0.000000,0.000000,0.000000
5,0.00000,0.000000,0.000000,0.00000,0.000000,0.888737,0.000000,0.000000
6,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.876855,0.000000
7,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.390133


In [ ]:
from scipy.stats.distributions import chi2
#проверка значимости полученной модели
#допустимо представление исходных признаков в виде m - факторной модели
mat1 = np.trace(A@np.linalg.inv(np.outer(eig_vec[0], eig_vec[0])+E_mat))
etta = - (n - 1 -1/6*(2*k+5)-2/3*m)*(np.log(np.linalg.det(A)/np.linalg.det( np.outer(eig_vec[0], eig_vec[0])+E_mat))-mat1 + k)
nu = 1/2*((k-m)**2-(k+m))
chi_2 = chi2.ppf(0.94, df=nu)
print('Критическое значение:', chi_2)
print(etta)
if etta<chi_2:
  print('Гипотеза H0 принимается')
else:
  print('Гипотеза H0 отвергается')

Критическое значение: 30.648863435677814
122.09245467143847
Гипотеза H0 отвергается


In [ ]:
def func(m, E_mat0, A ):
  eig_val, eig_vec = np.linalg.eig((A - E_mat0)@np.linalg.inv(E_mat0))
  eig_vec = eig_vec.T

  #Нормировка соственных векторов
  for i in range(m):
    c = (eig_vec[i].T@np.linalg.inv(E_mat0)@eig_vec[i])
    eig_vec[i]= eig_vec[i]*np.sqrt(np.abs(eig_val[i]/c))

  E_mat  = np.zeros((k,k))
  matrix =  np.sum(eig_vec[:m]**2, axis=0)
  diag_A = np.diag(A)
  _mat = diag_A - matrix
  for i in range(k):
    E_mat[i,i] = _mat[i]
  return E_mat0, E_mat , eig_vec

D_i = np.var(data_increment, axis=1, ddof=1)
mat = (np.ones(k)-np.abs(r_i_mat))*D_i
E_mat0 = np.zeros((k, k))
for i in range(k):
  E_mat0[i, i]=mat[i]
print(np.abs(np.trace(E_mat)-np.trace(E_mat0)))

while np.abs(np.trace(E_mat)-np.trace(E_mat0))>0.01:
  E_mat0 = E_mat
  E_mat0, E_mat, eig_vec = func(2,E_mat0, A )
  print(np.abs(np.trace(E_mat)-np.trace(E_mat0)))

pd.DataFrame(E_mat)

1.8860956140966136
0.9488888181212207
0.19692739716630747
0.053112613981069146
0.019144765824131937
0.008503707943598116


,0,1,2,3,4,5,6,7
0,0.83561,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
1,0.00000,0.804208,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
2,0.00000,0.000000,0.692565,0.000000,0.000000,0.00000,0.000000,0.00000
3,0.00000,0.000000,0.000000,0.307572,0.000000,0.00000,0.000000,0.00000
4,0.00000,0.000000,0.000000,0.000000,0.833807,0.00000,0.000000,0.00000
5,0.00000,0.000000,0.000000,0.000000,0.000000,0.53066,0.000000,0.00000
6,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.701769,0.00000
7,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.32478


In [ ]:
from scipy.stats.distributions import chi2
#проверка значимости полученной модели
#Гипотеза о возможности представления исходного вектора признаков с помощью модели факторного анализа  с m факторами
def model_check(m, k, n, A, eig_vec, E_mat):
  mat1 = np.trace(A@np.linalg.inv(eig_vec[:m].T@eig_vec[:m]+E_mat))
  mat2 = np.linalg.det(A)
  mat3 = np.linalg.det( eig_vec[:m].T@eig_vec[:m] + E_mat)
  etta = - (n - 1 -1/6*(2*k+5)-2/3*m)*(np.log(mat2/mat3)-mat1 + k)
  nu = 1/2*((k-m)**2-(k+m))
  chi_2 = chi2.ppf(0.94, df=nu)
  print('Значение статистики:', etta)
  print('Степени свободы:', nu)
  print('Критическое значение:', chi_2)
  if etta<chi_2:
    print('Гипотеза H0 принимается')
  else:
    print('Гипотеза H0 отвергается')
model_check(2, k, n, A, eig_vec, E_mat)

Значение статистики: 34.297513272817135
Степени свободы: 13.0
Критическое значение: 21.711276082539463
Гипотеза H0 отвергается


In [ ]:
D_i = np.var(data_increment, axis=1, ddof=1)
mat = (np.ones(k)-np.abs(r_i_mat))*D_i
E_mat0 = np.zeros((k, k))
for i in range(k):
  E_mat0[i, i]=mat[i]
print(np.abs(np.trace(E_mat)-np.trace(E_mat0)))

while np.abs(np.trace(E_mat)-np.trace(E_mat0))>0.001:
  E_mat0 = E_mat
  E_mat0, E_mat, eig_vec = func(3,E_mat0, A )


pd.DataFrame(E_mat)

0.4096300635922443


,0,1,2,3,4,5,6,7
0,0.82605,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,0.00000,0.794404,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,0.00000,0.000000,0.283657,0.000000,0.000000,0.00000,0.000000,0.000000
3,0.00000,0.000000,0.000000,0.394925,0.000000,0.00000,0.000000,0.000000
4,0.00000,0.000000,0.000000,0.000000,0.813113,0.00000,0.000000,0.000000
5,0.00000,0.000000,0.000000,0.000000,0.000000,0.26945,0.000000,0.000000
6,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.704361,0.000000
7,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.198578


In [ ]:
model_check(3, k, n, A, eig_vec, E_mat)

Значение статистики: 12.444412147984098
Степени свободы: 7.0
Критическое значение: 13.539733567440502
Гипотеза H0 принимается


In [ ]:
D_i = np.var(data_increment, axis=1, ddof=1)
mat = (np.ones(k)-np.abs(r_i_mat))*D_i
E_mat0 = np.zeros((k, k))
for i in range(k):
  E_mat0[i, i]=mat[i]
print(np.abs(np.trace(E_mat)-np.trace(E_mat0)))

while np.abs(np.trace(E_mat)-np.trace(E_mat0))>0.01:
  E_mat0 = E_mat
  E_mat0, E_mat, eig_vec = func(4,E_mat0, A )
  print(np.abs(np.trace(E_mat)-np.trace(E_mat0)))

0.08149198730001572
0.19574019825344013
0.07646934625387392
0.03271609418273247
0.015264442070114459
0.007947995532067598


In [ ]:
model_check(4, k, n, A, eig_vec, E_mat)


Значение статистики: 4.875388599209074
Степени свободы: 2.0
Критическое значение: 5.626821433520071
Гипотеза H0 принимается


In [ ]:
#доли дисперсий исходных признаков, объясняемые каждым обобщенным фактором
res = []
for i in range(k):
  p = []
  for j in range(4):
    l=np.sum(eig_vec[j, i]**2)/np.var(data_increment[i], ddof=1)# Доля дисперсии фактора Х{i+1}, объясняемая фактором {j+1}
    p.append(l)

  res.append(p)
res = np.array(res)
pd.DataFrame(res, ['X1', 'X2','X3', 'X4','X5','X6', 'X7', 'X8'], ['f1', 'f2', 'f3', 'f4'])

,f1,f2,f3,f4
X1,0.085622,0.058929,0.038995,0.062763
X2,0.126276,0.043630,0.056377,0.111801
X3,0.247167,0.221063,0.242590,0.001993
X4,0.470084,0.126141,0.007731,0.000957
X5,0.096238,0.056156,0.034001,0.013062
X6,0.300865,0.265150,0.158239,0.006917
X7,0.215421,0.086183,0.005809,0.079435
X8,0.561372,0.231502,0.001115,0.000119


In [ ]:
#Доли дисперсий исходных признаков, объясняемые совокупностью обобщенных факторов
for i in range(k):
  sm = np.sum(eig_vec[:m, i]**2)
  print(f'Доля дисперсии фактора Х{i+1}, объясняемая совокупностью факторов:  {sm/np.var(data_increment[i], ddof=1):.2f}')

Доля дисперсии фактора Х1, объясняемая совокупностью факторов:  0.09
Доля дисперсии фактора Х2, объясняемая совокупностью факторов:  0.13
Доля дисперсии фактора Х3, объясняемая совокупностью факторов:  0.25
Доля дисперсии фактора Х4, объясняемая совокупностью факторов:  0.47
Доля дисперсии фактора Х5, объясняемая совокупностью факторов:  0.10
Доля дисперсии фактора Х6, объясняемая совокупностью факторов:  0.30
Доля дисперсии фактора Х7, объясняемая совокупностью факторов:  0.22
Доля дисперсии фактора Х8, объясняемая совокупностью факторов:  0.56


In [ ]:
#Дисперсии характерных факторов
for i in range(k):
  print(f"Дисперсия {i+1}-го хаарктерного фактора: {E_mat[i,i]:.2f}")

Дисперсия 1-го хаарктерного фактора: 0.75
Дисперсия 2-го хаарктерного фактора: 0.66
Дисперсия 3-го хаарктерного фактора: 0.28
Дисперсия 4-го хаарктерного фактора: 0.39
Дисперсия 5-го хаарктерного фактора: 0.80
Дисперсия 6-го хаарктерного фактора: 0.26
Дисперсия 7-го хаарктерного фактора: 0.61
Дисперсия 8-го хаарктерного фактора: 0.20


In [ ]:
#Разложение для исходных признаков через обощенные и характерные факторы
for i in range(8):
    eq = f"X{i+1} = "
    parts = []
    for j in range(4):
        w = eig_vec[i, j]
        parts.append(f"{w:.3f}·F{j+1}")
    eq += " + ".join(parts) + f" + ε{i+1}"
    print(eq)

X1 = -0.294·F1 + -0.356·F2 + -0.499·F3 + -0.688·F4 + ε1
X2 = 0.243·F1 + 0.210·F2 + 0.472·F3 + -0.356·F4 + ε2
X3 = 0.198·F1 + 0.238·F2 + -0.494·F3 + -0.088·F4 + ε3
X4 = -0.251·F1 + -0.335·F2 + -0.045·F3 + 0.031·F4 + ε4
X5 = -0.552·F1 + 0.428·F2 + 0.148·F3 + -0.324·F4 + ε5
X6 = 0.338·F1 + 0.406·F2 + -0.062·F3 + 0.275·F4 + ε6
X7 = -0.566·F1 + 0.434·F2 + -0.148·F3 + 0.099·F4 + ε7
X8 = 0.378·F1 + -0.048·F2 + -0.085·F3 + -0.583·F4 + ε8


In [ ]:
#Метод Барлетта
#Найдем оценки факоров
mat = np.linalg.inv(eig_vec[:m]@np.linalg.inv(E_mat)@eig_vec[:m].T)
F = data_increment.T@np.linalg.inv(E_mat)@eig_vec[:m].T@mat
F.shape

(165, 1)

In [ ]:

names = ['ROSN','AFLT','BANEP','VTBR','GAZP','GMNK','SIBN','LKON','MTSS',
         'MFON','MTLR','PLZL','RBSM','RUALR','SBER','SNGS','NVTK','TATN',
         'RTKM','ODVA','RASP']

Lambda = eig_vec[:, :4]  # ваши 4 фактора
[3, 4, 6, 13, 14, 16, 17, 20]

for j in range(4):
    # 3 самые сильные нагрузки
    top3_idx = np.argsort(np.abs(Lambda[:, j]))[-3:][::-1]

    print(f"\nФАКТОР {j+1}:")
    for i in top3_idx:
        load = Lambda[i, j]
        if abs(load) > 0.2:
            print(f"  {'+' if load>0 else '-'} {names[i]} ({abs(load):.2f})")

    # Угадываем
    top_stocks = [names[i] for i in top3_idx if abs(Lambda[i,j])>0.2]

    if any(x in ['SBER','VTBR','BANEP'] for x in top_stocks):
        print("   БАНКОВСКИЙ ФАКТОР")
    elif any(x in ['GAZP','NVTK'] for x in top_stocks):
        print("   ГАЗОВЫЙ ФАКТОР")
    elif any(x in ['ROSN','LKON','SNGS','TATN'] for x in top_stocks):
        print("   НЕФТЯНОЙ ФАКТОР")
    elif any(x in ['GMNK','MTLR','RUALR','PLZL'] for x in top_stocks):
        print("   МЕТАЛЛУРГИЧЕСКИЙ ФАКТОР")
    elif any(x in ['MTSS','MFON','RTKM'] for x in top_stocks):
        print("   ТЕЛЕКОМ-ФАКТОР")
    else:
        print("  → СПЕЦИФИЧЕСКИЙ ФАКТОР")

print("\nИНТЕРПРЕТАЦИЯ:")
print("• F1: обычно 'общий рынок' (все акции вместе)")
print("• F2-F4: 'секторальные факторы' (банки/нефть/металлы)")
print("• Знак +: фактор растёт → акция растёт")
print("• Знак -: фактор растёт → акция падает")


ФАКТОР 1:
  - SIBN (0.57)
  - GAZP (0.55)
  + LKON (0.38)
   ГАЗОВЫЙ ФАКТОР

ФАКТОР 2:
  + SIBN (0.43)
  + GAZP (0.43)
  + GMNK (0.41)
   ГАЗОВЫЙ ФАКТОР

ФАКТОР 3:
  - ROSN (0.50)
  - BANEP (0.49)
  + AFLT (0.47)
   БАНКОВСКИЙ ФАКТОР

ФАКТОР 4:
  - ROSN (0.69)
  - LKON (0.58)
  - AFLT (0.36)
   НЕФТЯНОЙ ФАКТОР

ИНТЕРПРЕТАЦИЯ:
• F1: обычно 'общий рынок' (все акции вместе)
• F2-F4: 'секторальные факторы' (банки/нефть/металлы)
• Знак +: фактор растёт → акция растёт
• Знак -: фактор растёт → акция падает
